In [1]:
import pysam as ps
import numpy as np
import operator
import random
import timeit

In [2]:
tbx_glf = ps.TabixFile("loglikes.glf.gz")
tbx_mafs = ps.TabixFile("loglikes.mafs.gz")

In [22]:
def resize(resize_dim, pos_, gl, region_size):
    
    m = np.zeros((resize_dim, 3))
    for i, pos in enumerate(pos_):
        j = int(pos * resize_dim / region_size)
        m[j, :] += gl[i]
    
    return m

In [23]:
def choose_random(region, glf_gz, mafs_gz):
    
    # Chromesome list excluding x and y chromesomes
    chr_ = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
    
    # Array of number of bp in each crhomesome, from 1 - 22, minus region of choice
    chr_bp = np.array((248956422, 242193529, 198295559, 190214555, 181538259,
             170805979, 159345973, 145138636, 138394717, 133797422,
             135086622, 133275309, 114364328, 107043718, 101991189,
             90338345, 83257441, 80373285, 58617616, 64444167,
             46709983, 50818468)) - region
    
    # Chooses random chromesome and random starting position for the region
    chr_nr = random.randint(0, len(chr_)-1)
    start_pos = random.randint(0, chr_bp[chr_nr])
    
    # Indexing in the gz files via tabix through pysam
    glf = []
    mafs = []
    
    for i in glf_gz.fetch(chr_nr+1, start_pos, start_pos+region):
         glf.append(i.split('\t'))
            
    for i in mafs_gz.fetch(chr_nr+1, start_pos, start_pos+region):
         mafs.append(i.split('\t'))
    
    # Converts from strings to floats
    glf = np.float_(glf)
    
    # Splits into N individuals
    N = []
    for i in glf:
        N.append(np.array_split(i[2:], (len(i)-2)/10))
    
    # Looks up the major/minor alleles in the .mafs file and chooses the relevant genotypes likelihoods for
    # a random individual, and appends genomic positions to genomic_pos.
    gl_mat = []
    genomic_pos = []
    rand_N = random.randint(0,len(N[0])-1)
    for i, val in enumerate(N):
        if mafs[i][2] == 'A' and mafs[i][3] == 'C' or mafs[i][2] == 'C' and mafs[i][3] == 'A':
            gl_mat.append(np.array(operator.itemgetter(0,1,4)(val[rand_N])))
            genomic_pos.append(glf[i][1])
        elif mafs[i][2] == 'A' and mafs[i][3] == 'G' or mafs[i][2] == 'G' and mafs[i][3] == 'A':
            gl_mat.append(np.array(operator.itemgetter(0,2,7)(val[rand_N])))
            genomic_pos.append(glf[i][1])
        elif mafs[i][2] == 'A' and mafs[i][3] == 'T' or mafs[i][2] == 'T' and mafs[i][3] == 'A':
            gl_mat.append(np.array(operator.itemgetter(0,3,9)(val[rand_N])))
            genomic_pos.append(glf[i][1])
        elif mafs[i][2] == 'C' and mafs[i][3] == 'G' or mafs[i][2] == 'G' and mafs[i][3] == 'C':
            gl_mat.append(np.array(operator.itemgetter(4,5,7)(val[rand_N])))
            genomic_pos.append(glf[i][1])
        elif mafs[i][2] == 'C' and mafs[i][3] == 'T' or mafs[i][2] == 'T' and mafs[i][3] == 'C':
            gl_mat.append(np.array(operator.itemgetter(4,6,9)(val[rand_N])))
            genomic_pos.append(glf[i][1])
        elif mafs[i][2] == 'G' and mafs[i][3] == 'T' or mafs[i][2] == 'T' and mafs[i][3] == 'G':
            gl_mat.append(np.array(operator.itemgetter(7,8,9)(val[rand_N])))
            genomic_pos.append(glf[i][1])
        else:
            print('ERROR')
    
    count = 0
    for i in gl_mat:
        if sum(i) == 0:
            count += 1
    
    data_left = len(gl_mat) - count
    
    return chr_nr, start_pos, gl_mat, genomic_pos, data_left

In [24]:
def create_input(region, glf_gz, mafs_gz):
    
    data_left = 0
    threshold = 20
    
    while data_left < threshold:
        chr_nr, start_pos, gl_mat, genomic_pos, data_left = choose_random(region, glf_gz, mafs_gz)
    
    # locates relative genomic positions in region 
    rel_pos = np.asarray(genomic_pos) - start_pos
    
    # Resizes input to be a fixed size
    input_ = resize(128, rel_pos, gl_mat, region)
    
    return data_left, input_

In [37]:
start_time = timeit.default_timer()
data_left, input_ = create_input(4e+6, tbx_glf, tbx_mafs)
elapsed = timeit.default_timer() - start_time
elapsed

110.65264114085585

In [38]:
elapsed/60

1.8442106856809308

In [39]:
input_

array([[-119.27391 ,  -13.86312 ,  -50.887131],
       [ -41.216273,   -8.317872,  -57.33437 ],
       [ -33.387484,  -11.783652, -106.104215],
       [ -93.715215,   -9.704184,  -24.867919],
       [ -59.636955,  -14.556276, -117.201582],
       [ -75.294533,  -15.249432, -107.991242],
       [ -90.031077,  -19.408368, -142.760277],
       [ -59.406696,  -13.169964, -101.544004],
       [ -50.426614,   -9.704184,  -67.235486],
       [ -17.03913 ,   -4.158936,  -34.07826 ],
       [ -34.07826 ,   -6.238404,  -41.907049],
       [ -42.597825,   -8.317872,  -58.255403],
       [ -49.735838,   -9.011028,  -57.564627],
       [ -59.636955,  -10.39734 ,  -67.465744],
       [ -84.504874,  -15.249432, -101.544004],
       [ -84.504874,  -13.169964,  -75.985309],
       [ -59.636955,  -10.39734 ,  -67.465744],
       [-134.931488,  -18.022056,  -84.504874],
       [-118.583134,  -18.715212, -107.991241],
       [ -42.597825,   -9.704184,  -75.985309],
       [-110.754345,  -18.715212, -119.2

In [40]:
data_left

2058